In [ ]:
%reload_ext memory_profiler
import autograd.numpy as anp
from qoc import grape_schroedinger_discrete
from qoc.standard import TargetStateInfidelity
from memory_profiler import profile
import numpy as np

import matplotlib.pyplot as plt
from scipy.sparse import dia_matrix
%matplotlib inline

In [ ]:


# Define the system.
def get_memory_manually(N,l):
    """
    0_transmon_pi.py - This module demonstrates
    a simple example of grape on the schroedinger equation
    using time discrete control parameters to evolve a transmon qubit
    form the ground state to the first excited state.
    """


    HILBERT_SIZE = l
    # E.q. 19 (p. 6) of https://arxiv.org/abs/1904.06560.
    diagnol=np.arange(HILBERT_SIZE)
    up_diagnol = np.sqrt(diagnol)
    low_diagnol = np.sqrt(np.arange(1, HILBERT_SIZE + 1))
    state = (1 / np.sqrt(HILBERT_SIZE)) * np.ones(HILBERT_SIZE)
    data = [low_diagnol, diagnol, up_diagnol]
    offsets = [-1, 0, 1]
    H_SYSTEM_0= dia_matrix((data, offsets), shape=(HILBERT_SIZE, HILBERT_SIZE))
    sigmax = dia_matrix(([low_diagnol, up_diagnol], [-1, 1]), shape=(HILBERT_SIZE, HILBERT_SIZE))
    # Only real control amplitutdes are supported!
    hamiltonian = lambda controls, time: (H_SYSTEM_0
                                          + controls[0] * sigmax)

    # Define the problem.
    CAVITY_VACUUM = anp.zeros((l, 1))
    CAVITY_ZERO = anp.copy(CAVITY_VACUUM)
    CAVITY_ZERO[0, 0] = 1
    CAVITY_ONE = anp.copy(CAVITY_VACUUM)
    CAVITY_ONE[1, 0] = 1
    CAVITY_VACUUM= anp.zeros((l, 1))

    INITIAL_STATE_0=anp.copy(CAVITY_VACUUM)
    INITIAL_STATE_0[0,0]=1
    TARGET_STATE_0 = anp.copy(CAVITY_VACUUM)
    TARGET_STATE_0[1,0]=1
    INITIAL_STATES = anp.stack((INITIAL_STATE_0,), axis=0)
    TARGET_STATES = anp.stack((TARGET_STATE_0,), axis=0)
    COSTS = [TargetStateInfidelity(TARGET_STATES)]

    # Define the optimization.
    COMPLEX_CONTROLS = False
    CONTROL_COUNT = 1
    EVOLUTION_TIME = 1e-6  # nanoseconds
    CONTROL_EVAL_COUNT = SYSTEM_EVAL_COUNT = N + 1
    ITERATION_COUNT = 1

    # Define output.
    LOG_ITERATION_STEP = 1
    CONTROL_HAMILTONIAN = [sigmax]

    MANUAL_PARAMETER = {"control_hamiltonian": CONTROL_HAMILTONIAN,
                        "manual_gradient_mode": True, "tol": 1e-6}

    def main():
        result = grape_schroedinger_discrete(CONTROL_COUNT, CONTROL_EVAL_COUNT,
                                             COSTS, EVOLUTION_TIME, hamiltonian,
                                             INITIAL_STATES, SYSTEM_EVAL_COUNT,
                                             complex_controls=COMPLEX_CONTROLS,
                                             iteration_count=ITERATION_COUNT,
                                             log_iteration_step=LOG_ITERATION_STEP,
                                             manual_parameter=MANUAL_PARAMETER)
    if __name__ == "__main__":
        main()






In [ ]:
N_=1
#totol number of time step
mem_ma_Nfixed=[[],[]]
#first is average value,secind is standard deivation
h_dims=[1000000,2000000,3000000,4000000]
for dim in h_dims:
    mem_temp = %memit -r1 -i0.0001 -o get_memory_manually(N_,dim)
    mem_temp=np.array(mem_temp.mem_usage)
    mem_ma_Nfixed[0].append(np.average(mem_temp))
    mem_ma_Nfixed[1].append(np.std(mem_temp))
memory_plot(h_dims,mem_ma_Nfixed[0])
        

In [ ]:
def memory_plot(h_dim,mem_usage):
    fig, ax = plt.subplots()
    x = (np.array(h_dims))**1
    y = np.array(mem_usage)
    fit = np.polyfit(x,y,1)
    ang_coeff = fit[0]
    intercept = fit[1]
    maximum_ma=(122070.312 -intercept)/ang_coeff
    fit_eq = ang_coeff*x + intercept  
    ax.scatter(x,y,color='C0')
    ax.plot(x, y,color='C0', alpha = 0.5, label = 'new_manual')
    ax.set_xlabel("l")
    ax.set_ylabel("memory usage")
    print('AD '+"slope is "+str(ang_coeff)+"and intercept is "+str(intercept))
    print("Max h_dim is "+ str(maximum_ma))
    ax.legend()